In [1]:
!pip install pyarrow zstandard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 12.3 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [5]:
import pyarrow.feather as feather
import zstandard

import pyarrow.feather as feather
# CONTENT BASED EMBEDDINGS
# Specify the path to your .feather.zstd file
file_path = '/content/drive/Shareddrives/CSCE 470 Project/content.feather.zstd'

# Read the Feather file using pyarrow (which automatically handles decompression)
table = feather.read_table(file_path)

# Convert the table to a pandas DataFrame for easier viewing
all_content_embeds = table.to_pandas()

# Print or manipulate the DataFrame as needed
print(all_content_embeds)




                      channelId         0         1         2         3  \
0      UC--3c8RqSfAqYBdDjIG3UNA -0.015708  0.006428  0.020093  0.026353   
1      UC--3ozenTzry39-xMqAPQlw -0.043753  0.024361  0.109967 -0.034727   
2      UC--3pN-wtQrC48GAK8wsBsg -0.129836 -0.104198  0.071117 -0.153048   
3      UC--70ql_IxJmhmqXqrkJrWQ -0.049794  0.101294 -0.016527  0.011187   
4      UC--EwQJeJ6SKlJdswehQH4g -0.164854  0.170872  0.028569 -0.035195   
...                         ...       ...       ...       ...       ...   
43769  UCzzamhkOmpSnZ7BSBikvKnA -0.101376  0.088499  0.021926  0.043392   
43770  UCzzfu83LhPMMuhtDVA75rfw -0.025645  0.008495  0.040811  0.094434   
43771  UCzzsltdnAgTMSlPOvjko_Dw  0.054023  0.001200  0.034853 -0.003687   
43772  UCzztaOtjmaZoiLvWLagq7bg -0.151070 -0.006982 -0.007551 -0.055316   
43773  UCzzzUN8yvD2LRAnY-lhzyLQ -0.126983 -0.068605  0.033943 -0.045903   

              4         5         6         7         8  ...       374  \
0      0.022591  0.009369

In [8]:
#extract embeds of only our top 1000

chan_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chanIDs.csv'

chan_ids = pd.read_csv(chan_file_path)

merged_df = pd.merge(all_content_embeds, chan_ids, left_on='channelId', right_on='Youtube Url', how='inner')

# Drop the redundant column
merged_df = merged_df.drop(columns='Youtube Url')
merged_df

matching_chan_df = merged_df['channelId']

In [10]:
from sklearn.metrics.pairwise import cosine_similarity


In [7]:

# Function to get recommendations based on cosine similarity
def get_channel_recommendations(channel_id, embeddings_df, top_n=5):
    # Find the index of the given channel
    channel_index = embeddings_df.index[embeddings_df['channelId'] == channel_id].tolist()[0]

    # Select the embeddings for the given channel
    channel_embeddings = embeddings_df.iloc[channel_index, 1:]  # Assuming embeddings start from the second column

    # Reshape the embeddings to a 2D array
    channel_embeddings = channel_embeddings.values.reshape(1, -1)

    # Calculate cosine similarity between the given channel and all other channels
    similarity_scores = cosine_similarity(channel_embeddings, embeddings_df.iloc[:, 1:])

    # Get indices of top_n most similar channels (excluding the channel itself)
    similar_channel_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]

    # Get the channel_id of recommended channels
    recommended_channels = embeddings_df['channelId'].iloc[similar_channel_indices].tolist()

    return recommended_channels



In [11]:

chan_recommendations_dict = {}
for chan in matching_chan_df:
  recommendations = get_channel_recommendations(chan, merged_df)
  chan_recommendations_dict[chan] = recommendations
chan_recommendations_dict

{'UC-0D1RlSzcvBoODCKSoLl2g': ['UCzIdYMdAtTsWucGCZyZvN6w',
  'UCKqx9r4mrFglauNBJc1L_eg',
  'UC6D1L2vxEAg_Vi0JSxMBDgA',
  'UCblfuW_4rakIf2h6aqANefA',
  'UCe4LM_eKc9ywRmVuBm5pjQg'],
 'UC-2Y8dQb0S6DtpxNgAKoJKA': ['UCKy1dAqELo0zrOtPkf0eTMw',
  'UC2wKfjlioOCLP4xQMOWNcgg',
  'UC2t5bjwHdUX4vM2g8TRDq5g',
  'UCpqXJOEqGS-TCnazcHCo0rA',
  'UClG8odDC8TS6Zpqk9CGVQiQ'],
 'UC-Eh1lR_tWVtv9AvxLybMtQ': ['UClVrJwcIy7saPcGc1nct80A',
  'UCMJ5Qf3sOvQpcYiai1Noa3Q',
  'UCpDJl2EmP7Oh90Vylx0dZtA',
  'UClZF7OsrECe3t_ADrjZEh_Q',
  'UCEdvpU2pFRCVqU6yIPyTpMQ'],
 'UC-SV8-bUJfXjrRMnp7F8Wzw': ['UCvthuVsurPaVz2a7_4LepGg',
  'UCja7QUMRG9AD8X2F_vXFb9A',
  'UCKaCalz5N5ienIbfPzEbYuA',
  'UC70Dib4MvFfT1tU6MqeyHpQ',
  'UCp0hYYBW6IMayGgR-WeoCvQ'],
 'UC-lHJZR3Gqxm24_Vd_AJ5Yw': ['UC_hSjGOO67A6gIDUcyuv_8A',
  'UCMiJRAwDNSNzuYeN2uWa0pA',
  'UCx6F-rETGiz7xf_vkMmX2yQ',
  'UCw1SQ6QRRtfAhrN_cjkrOgA',
  'UCRijo3ddMTht_IHyNSNXpNQ'],
 'UC-yXuc1__OzjwpsJPlxYUCQ': ['UCGt7X90Au6BV8rf49BiM6Dg',
  'UCa10nxShhzNrCE1o2ZOPztg',
  'UCplkk3J5wrEl0

In [12]:
# correlate chan ids to chan names
chan_names_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chan_names_IDs.csv'

chan_names_ids = pd.read_csv(chan_names_file_path)

chan_usernames_dict = {}
# Traverse the chan_recommendations_dict and correlate IDs to names
for chan_id, recommendations in chan_recommendations_dict.items():
    # Find the corresponding username for the channel ID
    matching_row = chan_names_ids[chan_names_ids['Youtube Url'] == chan_id]
    matching_row = matching_row['username'].values[0]
    chan_usernames_dict[matching_row] = []
    for rec in recommendations:
      #translate the recs too
      translated_rec = chan_names_ids[chan_names_ids['Youtube Url'] == rec]
      chan_usernames_dict[matching_row].append(translated_rec['username'].values[0])

# Now chan_usernames_dict contains the mapping of channel IDs to usernames
print(chan_usernames_dict)


{'Emily Tube': ['Nat and Essie', 'toypudding', 'Dan Rhodes', 'redbull', 'AtlanticVideos'], 'PlayStation': ['IGNentertainment', 'typicalgamer', 'RiotGamesInc', 'theRadBrad', 'epicfortnite'], 'TygaTygaTV': ['FueledByRamen', 'majorlazer', 'SpinninRec', 'Geet MP3', 'Marshmello'], 'RomanAtwoodVlogs': ['familyfunpack', 'The Royalty Family', 'jordanmatter', 'PrestonPlayz', 'TheEllenShow'], 'PewDiePie': ['Infinite', 'Mrwhosetheboss', 'Mythpat', 'LazarBeam', 'corycotton'], 'WorldStarHipHopTV': ['RayWilliamJohnson', 'AllTrapNation', 'Kurt Hugo Schneider', 'BuzzFeedVideo', 'Davie504'], 'Crafty Panda': ['123 GO!', 'Avocado Couple I Crazy Comics', '5-Minute Crafts LIKE', 'Troom Troom', '5-Minute Crafts DIY'], 'DJ Khaled': ['cleanbandit', 'Meghan Trainor', 'FueledByRamen', 'Kurt Hugo Schneider', 'SpinninRec'], 'Doctor Mike': ['The Daily Show with Trevor Noah', 'collegehumor', 'Beast Reacts', 'BuzzFeedVideo', 'barcroftmedia'], 'xoxxxoooxo': ['FueledByRamen', 'SpinninRec', 'MrSuicideSheep', 'DisneyMus

In [13]:
df_channels_usernames = pd.DataFrame(list(chan_usernames_dict.items()), columns=['Channel', 'Recommendations'])

# Save the DataFrame to a CSV file
csv_file_path = 'content_recs.csv'
df_channels_usernames.to_csv(csv_file_path, index=False)

Didnt work for James Charles, and TedX Talks channels. Gave error (probably not in the 40,000 channels)